### ACS IPUMS NHGIS - Basic Data Preparation

In [1]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


  Cloning git://github.com/geopandas/geopandas.git to c:\users\sheld\appdata\local\temp\pip-req-build-cxykp3tc
  Created wheel for geopandas: filename=geopandas-0.8.0+98.gefea225-py2.py3-none-any.whl size=981829 sha256=912248583ceb2a6dc4d5cf81a3d772e18b21e483b9d68e3b00c0099b7ed09a4f
  Stored in directory: C:\Users\sheld\AppData\Local\Temp\pip-ephem-wheel-cache-0s05hmba\wheels\cf\3e\0b\6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas


  Running command git clone -q git://github.com/geopandas/geopandas.git 'C:\Users\sheld\AppData\Local\Temp\pip-req-build-cxykp3tc'


Wall time: 1min 3s


In [1]:
import pandas as pd
import numpy as np
import re
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon, LineString
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
def import_csv(content):
    """Read in csv as a dataframe"""
    acs_data = pd.read_csv(content, encoding= 'unicode_escape')
    return acs_data

In [3]:
def rename_variables(dataframe, dictionary):
    """Renames columns of a dataframe given dictionary"""
    dataframe = dataframe.rename(columns = dictionary)
    return dataframe

In [4]:
def strip_dict_keys_values(d):
    """To remove white-space in keys and values in dictionary"""
    def strip_list(l):
        return [strip_dict_keys_values(x)
                if isinstance(x, dict) else strip_list(x)
                if isinstance(x, list) else clean(x) for x in l]

    def clean(value):
        if isinstance(value, str):
            return value.strip()
        return value

    return {key.strip(): strip_dict_keys_values(value)
            if isinstance(value, dict) else strip_list(value)
            if isinstance(value, list) else clean(value)
            if value is None else clean(value)
            for key, value in d.items()}

In [5]:
def block_groups(list_bg_data):
    """Takes in list of block group shapefiles return files concat together for 11 states."""
    block_group_shp = [0]*11
    for i in range(len(list_bg_data)):
        block_group_shp[i] = gpd.read_file(list_bg_data[i])

    block_group_con = pd.concat(block_group_shp, axis = 0)
    block_group_con = block_group_con.reset_index()
    
    return block_group_con

In [6]:
def block_groups_us(bg_data):
    """Reads in shp files for all US block groups"""
    block_group_con = gpd.read_file(bg_data)
    return block_group_con

In [7]:
def cpi_adjustment(Year_1, Year_2, data_CPI):
    """Calculates Inflation Adjustment Factor for dollar denominated variables"""
    """Uses  R-CPI-U-RS, All items CPI estimates from https://www.bls.gov/cpi/research-series/r-cpi-u-rs-home.htm"""
    CPI_i = data_CPI.set_index("ï»¿YEAR")
    CPI_p2 = CPI_i.at[Year_2,"AVG"]
    CPI_p1 = CPI_i.at[Year_1,"AVG"]
    Inflation_Adjustment_Factor = CPI_p2/CPI_p1
    return Inflation_Adjustment_Factor


In [8]:
ACS5_2009 = import_csv(r"C:\Users\sheld\Documents\nhgis0005_csv\nhgis0005_ds195_20095_2009_blck_grp.csv")

In [9]:
ACS5_2019 = import_csv(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis0003_csv\nhgis0003_ds244_20195_2019_blck_grp.csv')

In [10]:
ACS5_2014 = import_csv(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis0004_csv\nhgis0004_ds206_20145_2014_blck_grp.csv')

In [11]:
# ACS5_2019_names = import_csv(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis_2019_varaible_names.csv')
# ACS5_2014_names = import_csv(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis_2014_variable_names.csv')
# ACS5_2009_names = import_csv(r"C:\Users\sheld\Documents\nhgis0005_csv\nhgis_2009_variable_names.csv")

ACS5_2019_names = import_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis_2019_varaible_names_Updated.csv")
ACS5_2014_names = import_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis_2014_variable_names_Updated.csv")
ACS5_2009_names = import_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis_2009_variable_names_Updated.csv")

In [12]:
CPI = import_csv(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\r-cpi-u-rs-allitems (1).csv')

Code below to rename variables from GIS names. 

In [13]:
# ACS5_2019_names = ACS5_2019_names.drop(columns = ['1','2','3','4','5','6','7','8','9'])
# ACS5_2014_names = ACS5_2014_names.drop(columns = ['1','2','3','4','5','6','7','8','9'])

In [14]:
dictionary_2019 = dict(ACS5_2019_names.values.tolist())
dictionary_2014 = dict(ACS5_2014_names.values.tolist())
dictionary_2009 = dict(ACS5_2009_names.values.tolist())

In [15]:
dictionary_2019_new = strip_dict_keys_values(dictionary_2019) # remove whitespace from dictionary. 
dictionary_2014_new = strip_dict_keys_values(dictionary_2014) # remove whitespace from dictionary. 
dictionary_2009_new = strip_dict_keys_values(dictionary_2009)

In [16]:
# ACS5_2019_new = rename_variables(ACS5_2019, dictionary_2019_new) #rename variables, variables with ending in M are margins of error. 
# ACS5_2014_new = rename_variables(ACS5_2014, dictionary_2014_new)
# ACS5_2009_new = rename_variables(ACS5_2009, dictionary_2009_new)

In [17]:
# block_groups_2019 = ['/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_040_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_060_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_080_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_160_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_300_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_320_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_350_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_410_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_490_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_530_blck_grp_2019.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2019_nhgis0002_shape/nhgis0002_shapefile_tl2019_560_blck_grp_2019.zip']
# block_groups_2014 = ['/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_040_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_060_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_080_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_160_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_300_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_320_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_350_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_410_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_490_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_530_blck_grp_2014.zip','/content/drive/MyDrive/Colab Notebooks/ACS_IPUMS_NHGIS_Data/ACS5_2014_nhgis0001_shape/nhgis0001_shapefile_tl2014_560_blck_grp_2014.zip']

In [18]:
# ACS5_2019_block_groups = block_groups(block_groups_2019)
# ACS5_2014_block_groups = block_groups(block_groups_2014)

In [19]:
ACS5_2019_block_groups = block_groups_us(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis0003_shape\nhgis0003_shapefile_tl2019_us_blck_grp_2019.zip')
ACS5_2014_block_groups = block_groups_us(r'C:\Users\sheld\Documents\Thesis_Data_Full_United_States\nhgis0004_shape\nhgis0004_shapefile_tl2014_us_blck_grp_2014.zip')
ACS5_2009_block_groups = block_groups_us(r'C:\Users\sheld\Documents\nhgis0006_shape\nhgis0006_shapefile_tl2010_us_blck_grp_2010.zip')

In [20]:
ACS5_2019_bg = pd.merge(ACS5_2019, ACS5_2019_block_groups, on = ['GISJOIN'], how = "left") #Joining block groups with ACS5 data. 
ACS5_2014_bg = pd.merge(ACS5_2014, ACS5_2014_block_groups, on = ['GISJOIN'], how = "left")
ACS5_2009_bg = pd.merge(ACS5_2009, ACS5_2009_block_groups, on = ['GISJOIN'], how = "left") #-- Had to droop 25% of obs in doing calculations of min dist so trying with 2014 bg for 2009 instead of 2009 bg's --> Turned out to be workse 59,999 missings obs so I am sticking to 2009 bg for 2009 acs 
#ACS5_2009_bg = pd.merge(ACS5_2009, ACS5_2014_block_groups, on = ['GISJOIN'], how = "left") #here try with 2014 bg for 2009

Inflation Adjust the Dollar Amount Multiyear Estimates (2010-2014) to Compare in 2019 Dollars where the inflation adjusted estimate for the period earlier can be expressed as:

\begin{equation}
  \hat{X}_{p1,Adj} = \frac{CPI_{p2}}{CPI_{p1}} \hat{X}_{p1}
\end{equation}

$CPI_{p1}$ - is the All Items CPI-U-RS Annual Average for the last year in the earlier time period (P1). 

$CPI_{p2}$ -  is the All Items CPI-U-RS Annual Average for the last year in the most current time period (P2). 

$\hat{X}_{p1}$ - is the published ACS estimate for the earlier time period (P1).

Documentation on inflation adjusting dollar estimates can be found [here]("https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch10.pdf"). 

[R-CPI-U-RS from the U.S. Bureau of Labor Statistics]("https://www.bls.gov/cpi/research-series/r-cpi-u-rs-home.htm").


In [21]:
Inflation_Adjustment_2014 = cpi_adjustment(2014, 2019, CPI)
Inflation_Adjustment_2009 = cpi_adjustment(2009, 2019, CPI)

In [22]:
dollar_variables = ["ABIHE001", "ABISE001", "ABITE001", "ABIUE001"] #inflation adjusted 2014 dollar variables (i.e. rent and lower, median, upper household value) into 2019 dollars 
for i in range(len(dollar_variables)):
    ACS5_2014_bg[dollar_variables[i]] = Inflation_Adjustment_2014*ACS5_2014_bg[dollar_variables[i]]

In [23]:
dollar_variables = ["RRUE001","RR6E001", "RR7E001", "RR8E001"] #inflation adjusted 2009 dollar variables (i.e. rent and lower, median, upper household value) into 2019 dollars 
for i in range(len(dollar_variables)):
    ACS5_2009_bg[dollar_variables[i]] = Inflation_Adjustment_2009*ACS5_2009_bg[dollar_variables[i]]

In [24]:
# ACS5_2014_bg.to_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\merged_data_with_block_groups\ACS5_2014_bg.csv")
# ACS5_2019_bg.to_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\merged_data_with_block_groups\ACS5_2019_bg.csv")
# ACS5_2009_bg.to_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\merged_data_with_block_groups\ACS5_2009_bg.csv")

In [25]:
ACS5_2009_bg = ACS5_2009_bg.rename(columns={"ALAND10":"ALAND", "AWATER10":"AWATER","INTPTLAT10":"INTPTLAT","INTPTLON10":"INTPTLON","Shape_area":"Shape_Area","Shape_len":"Shape_Leng"})

### Organizing 2005-2009 ACS 

In [26]:
cols = list(ACS5_2009_bg.columns.values)[146:180]
for i in range(17):
    ACS5_2009_bg[f"educ{i}"] = ACS5_2009_bg[cols[i]]+ACS5_2009_bg[cols[i+17]]
#adding male and female categories to get total for education categories for 2005-2009 data
#NOTE! remember to drop cols 146 to 180-1. ******

### Combine Year Structure Built Variable for Later Period ACS 2010-2014 and 2015-2019

In [27]:
ACS5_2019_bg["built_2000"] = ACS5_2019_bg["AL0DE004"]+ACS5_2019_bg["AL0DE003"]+ACS5_2019_bg["AL0DE002"]
#combined Built 2000 to 2009, 2009 to 2013, and 2014 or later 
ACS5_2014_bg["built_2000"] = ACS5_2014_bg["ABHPE003"]+ACS5_2014_bg["ABHPE002"]
#combined Built 2000 and 2010 or later for 2014 
ACS5_2009_bg["built_2000"] = ACS5_2009_bg["RQ2M002"] + ACS5_2009_bg["RQ2M003"]
#combined built 2000-2004 and 2005 or later for 2009

#NOTE! remeber to drop old columns (e.g. RQ2M002, etc)

In [28]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.reset_option('^display.', silent=True)

In [29]:
#Nursery to grade 4
ACS5_2019_bg["educ2"] = ACS5_2019_bg["ALWGE003"]+ACS5_2019_bg["ALWGE004"]+ACS5_2019_bg["ALWGE005"]+ACS5_2019_bg["ALWGE006"]+ACS5_2019_bg["ALWGE007"]+ ACS5_2019_bg["ALWGE008"]
ACS5_2014_bg["educ2"] = ACS5_2014_bg["ABC4E003"] + ACS5_2014_bg["ABC4E004"]+ ACS5_2014_bg["ABC4E005"]+ ACS5_2014_bg["ABC4E006"]+ACS5_2014_bg["ABC4E007"]+ACS5_2014_bg["ABC4E008"]

#grades five and 6 
ACS5_2019_bg["educ3"] = ACS5_2019_bg["ALWGE009"]+ACS5_2019_bg["ALWGE010"]
ACS5_2014_bg["educ3"] = ACS5_2014_bg["ABC4E009"]+ACS5_2014_bg["ABC4E010"]

#grades seven and 8 
ACS5_2019_bg["educ4"] = ACS5_2019_bg["ALWGE011"]+ACS5_2019_bg["ALWGE012"]
ACS5_2014_bg["educ4"] = ACS5_2014_bg["ABC4E011"]+ACS5_2014_bg["ABC4E012"]

#regular hs and GED combined 
ACS5_2019_bg["educ9"] = ACS5_2019_bg["ALWGE017"]+ACS5_2019_bg["ALWGE018"]
ACS5_2014_bg["educ9"] =ACS5_2014_bg["ABC4E017"]+ACS5_2014_bg["ABC4E018"]


In [30]:
ACS5_2019_new = rename_variables(ACS5_2019_bg, dictionary_2019_new) #rename variables for merging df
ACS5_2014_new = rename_variables(ACS5_2014_bg, dictionary_2014_new)
ACS5_2009_new = rename_variables(ACS5_2009_bg, dictionary_2009_new)

In [31]:
#ACS5_2009_new.columns.tolist() NOTE! For 2009 bg shp will have to rename lat and long - remove 10!!!Look for other cols to rename if necessary =

In [32]:
frames = [ACS5_2019_new, ACS5_2014_new, ACS5_2009_new]
ACS5_2005_2019 = pd.concat(frames,join='inner', ignore_index=True).reset_index()

In [33]:
cols = ['index',
 'GISJOIN',
 'YEAR',
 'REGIONA',
 'DIVISIONA',
 'STATE',
 'STATEA',
 'COUNTY',
 'COUNTYA',
 'COUSUBA',
 'PLACEA',
 'TRACTA',
 'BLKGRPA',
 'CONCITA',
 'AIANHHA',
 'RES_ONLYA',
 'TRUSTA',
 'ANRCA',
 'CBSAA',
 'CSAA',
 'METDIVA',
 'NECTAA',
 'CNECTAA',
 'NECTADIVA',
 'UAA',
 'CDCURRA',
 'SLDUA',
 'SLDLA',
 'SUBMCDA',
 'SDELMA',
 'SDSECA',
 'SDUNIA',
 'PUMA5A',
 'NAME_E',
 'Total Race',
 'White alone',
 'Black or African American alone',
 'American Indian and Alaska Native alone',
 'Asian alone',
 'Native Hawaiian and Other Pacific Islander alone',
 'Some other race alone',
 'Two or more races',
 'Two or more races Two races including Some other race',
 'Two or more races Two races excluding Some other race and three or more races',
 'Total Travel Time to Work',
 'Less than 5 minutes',
 '5 to 9 minutes',
 '10 to 14 minutes',
 '15 to 19 minutes',
 '20 to 24 minutes',
 '25 to 29 minutes',
 '30 to 34 minutes',
 '35 to 39 minutes',
 '40 to 44 minutes',
 '45 to 59 minutes',
 '60 to 89 minutes',
 '90 or more minutes',
 'Total Educational Attainment for the Population 25 Years and Over',
 'No schooling completed',
 'Nursery to 4th grade',
 '5th and 6th grade',
 '7th and 8th grade',
 '9th grade',
 '10th grade',
 '11th grade',
 '12th grade no diploma',
 'High school graduate, GED, or alternative',
 'Some college less than 1 year',
 'Some college 1 or more years no degree',
 "Associate's degree",
 "Bachelor's degree",
 "Master's degree",
 'Professional school degree',
 'Doctorate degree',
 'Total Year Structure Built',
 'Built 2000 or later',
 'Built 1990 to 1999',
 'Built 1980 to 1989',
 'Built 1970 to 1979',
 'Built 1960 to 1969',
 'Built 1950 to 1959',
 'Built 1940 to 1949',
 'Built 1939 or earlier',
 'Total Bedrooms',
 'No bedroom',
 '1 bedroom',
 '2 bedrooms',
 '3 bedrooms',
 '4 bedrooms',
 '5 or more bedrooms',
 'Median gross rent',
 'Lower value quartile (dollars)',
 'Median value (dollars)',
 'Upper value quartile (dollars)',
 'Total Mortgage Status',
 'Housing units with a mortgage contract to purchase or similar debt',
 'Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both',
 'Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both Second mortgage only',
 'Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both Home equity loan only',
 'Housing units with a mortgage contract to purchase or similar debt Both second mortgage and home equity loan',
 'Housing units with a mortgage contract to purchase or similar debt No second mortgage and no home equity loan',
 'Housing units without a mortgage',
 'NAME_M',
 'ALAND',
 'AWATER',
 'INTPTLAT',
 'INTPTLON',
 'Shape_Leng',
 'Shape_Area',
 'geometry']

In [34]:
ACS5_2005_2019 = ACS5_2005_2019[cols]

In [35]:
remove = ['AIANHHA',
 'RES_ONLYA',
 'TRUSTA',
 'ANRCA',
 'CBSAA',
 'CSAA',
 'METDIVA',
 'NECTAA',
 'CNECTAA',
 'NECTADIVA',
 'UAA',
 'CDCURRA',
 'SLDUA',
 'SLDLA',
 'SUBMCDA',
 'SDELMA',
 'SDSECA',
 'SDUNIA',
 'PUMA5A',
 'NAME_M']

In [36]:
ACS5_2005_2019 = ACS5_2005_2019.drop(columns=remove)

## Get Data Into % Format

In [39]:
cols = ["Two or more races Two races including Some other race","Two or more races Two races excluding Some other race and three or more races"]
ACS5_2005_2019 = ACS5_2005_2019.drop(columns=cols) # `Two or more races' already captures this variable so these extra two are not needed

In [45]:
#get race in % formate
cols = list(ACS5_2005_2019.columns.values)[16:23]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Race"]

In [52]:
#get travel time to work in % formate
cols = list(ACS5_2005_2019.columns.values)[24:36]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Travel Time to Work"]

In [57]:
#get education into % formate
cols = list(ACS5_2005_2019.columns.values)[37:53]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Educational Attainment for the Population 25 Years and Over"]

In [62]:
#get year struture built in % formate
cols = list(ACS5_2005_2019.columns.values)[54:62]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Year Structure Built"]

In [67]:
#get total bedrooms in % formate
cols = list(ACS5_2005_2019.columns.values)[63:69]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Bedrooms"]

In [70]:
cols = ["Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both Second mortgage only","Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both Home equity loan only"]
ACS5_2005_2019 = ACS5_2005_2019.drop(columns=cols) #droping sub categories under `Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both' note that 
#`Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both' + `Housing units with a mortgage contract to purchase or similar debt Both second mortgage and home equity loan'+`Housing units with a mortgage contract to purchase or similar debt No second mortgage and no home equity loan' = `Housing units with a mortgage contract to purchase or similar debt' 

In [75]:
#get mortgage status in % formate 
cols = list(ACS5_2005_2019.columns.values)[74:79]
for i in range(len(cols)):
    ACS5_2005_2019[cols[i]]=ACS5_2005_2019[cols[i]]/ACS5_2005_2019["Total Mortgage Status"]

### Join Complete DF with Min Dist Wildfires for Each Year

In [81]:
min_dist_2005_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2005_ACS52009_sim.csv")
min_dist_2006_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2006_ACS52009_sim.csv")
min_dist_2007_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2007_ACS52009_sim.csv")
min_dist_2008_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2008_ACS52009_sim.csv")
min_dist_2009_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2009_ACS52009_sim.csv")
min_dist_2010_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2010_ACS52014_sim.csv")
min_dist_2011_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2011_ACS52014_sim.csv")
min_dist_2012_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2012_ACS52014_sim.csv")
min_dist_2013_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2013_ACS52014_sim.csv")
min_dist_2014_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2014_ACS52014_sim.csv")
min_dist_2015_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2015_ACS52014_sim.csv")
min_dist_2016_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2016_ACS52014_sim.csv")
min_dist_2017_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2017_ACS52014_sim.csv")
min_dist_2018_sim = import_csv(r"C:\Users\sheld\Documents\GitHub\econ499\Min_distances_2018_ACS52014_sim.csv")
#Note these are only the simpify shapes, need to re-calculate 2005-2009 box shapes for those years

In [86]:
dist = [min_dist_2005_sim, min_dist_2006_sim, min_dist_2007_sim, min_dist_2008_sim, min_dist_2009_sim, min_dist_2010_sim, min_dist_2011_sim, min_dist_2012_sim, min_dist_2013_sim, min_dist_2014_sim, min_dist_2015_sim, min_dist_2016_sim, min_dist_2017_sim, min_dist_2018_sim]

In [87]:
#remove white space on GISJOIN variable for all distance series. 
for i in range(len(dist)):
    dist[i]["GISJOIN"] = dist[i]["GISJOIN"].str.replace(' ','')

In [88]:
def merge_on_dist(ACS, dist):
    for i in range(len(dist)):
        ACS = pd.merge(ACS, dist[i], on="GISJOIN")
    return ACS

In [98]:
ACS5_2005_2019_WEST = ACS5_2005_2019[(ACS5_2005_2019["STATE"] == "Arizona") | (ACS5_2005_2019["STATE"] == "California")|(ACS5_2005_2019["STATE"] == "Colorado") | (ACS5_2005_2019["STATE"] == "Idaho")|(ACS5_2005_2019["STATE"] == "Nebraska")| (ACS5_2005_2019["STATE"] == "New Mexico")|(ACS5_2005_2019["STATE"] == "Oklahoma")| (ACS5_2005_2019["STATE"] == "Oregon")|(ACS5_2005_2019["STATE"] == "South Dakota")|(ACS5_2005_2019["STATE"] == "Texas")|(ACS5_2005_2019["STATE"] == "Kansas")| (ACS5_2005_2019["STATE"] == "Wyoming")|(ACS5_2005_2019["STATE"] == "Montana")|(ACS5_2005_2019["STATE"] == "North Dakota")|(ACS5_2005_2019["STATE"] == "Utah")|(ACS5_2005_2019["STATE"] == "Nevada")| (ACS5_2005_2019["STATE"] == "Washington")]

In [100]:
ACS5_2005_2019_WESTd = merge_on_dist(ACS5_2005_2019_WEST, dist) #merging min wildfire distances

,index,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,min_dist_2009_sim,min_dist_2010_sim,min_dist_2011_sim,min_dist_2012_sim,min_dist_2013_sim,min_dist_2014_sim,min_dist_2015_sim,min_dist_2016_sim,min_dist_2017_sim,min_dist_2018_sim
0,3972,G04000109426001,2015-2019,NaN,NaN,Arizona,4,Apache County,1,NaN,...,120782.03740,94759.52439,115027.99880,25503.269930,125270.03560,85914.48854,111499.68000,133972.656000,100893.33770,107968.383600
1,224305,G04000109426001,2010-2014,NaN,NaN,Arizona,4,Apache County,1,NaN,...,120782.03740,94759.52439,115027.99880,25503.269930,125270.03560,85914.48854,111499.68000,133972.656000,100893.33770,107968.383600
2,444530,G04000109426001,2005-2009,NaN,NaN,Arizona,4,Apache County,1,NaN,...,120782.03740,94759.52439,115027.99880,25503.269930,125270.03560,85914.48854,111499.68000,133972.656000,100893.33770,107968.383600
3,3974,G04000109427001,2015-2019,NaN,NaN,Arizona,4,Apache County,1,NaN,...,69603.85231,74790.55545,78663.90506,49846.002650,102141.75150,98346.86015,48466.86904,83506.334860,92389.39464,71507.596710
4,224307,G04000109427001,2010-2014,NaN,NaN,Arizona,4,Apache County,1,NaN,...,69603.85231,74790.55545,78663.90506,49846.002650,102141.75150,98346.86015,48466.86904,83506.334860,92389.39464,71507.596710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131165,649462,G56004509513003,2005-2009,NaN,NaN,Wyoming,56,Weston County,45,NaN,...,139152.18100,60489.72362,11662.59376,8275.394046,89855.95756,210002.30730,25171.56196,8254.181101,54241.85561,8043.027649
131166,399242,G46011309405001,2010-2014,NaN,NaN,South Dakota,46,Shannon County,113,NaN,...,275581.09980,84938.01376,86864.30068,17649.409310,222388.97080,337930.78760,68052.81444,25872.515480,67201.38599,138345.342900
131167,612941,G46011309405001,2005-2009,NaN,NaN,South Dakota,46,Shannon County,113,NaN,...,275581.09980,84938.01376,86864.30068,17649.409310,222388.97080,337930.78760,68052.81444,25872.515480,67201.38599,138345.342900
131168,399243,G46011309405002,2010-2014,NaN,NaN,South Dakota,46,Shannon County,113,NaN,...,300336.90930,109498.31900,115809.95600,12843.476690,246677.65790,362573.24790,93778.61921,47340.692400,90864.18991,166084.585700


In [114]:
missing_house_prices = ACS5_2005_2019_WESTd[ACS5_2005_2019_WESTd["Median value (dollars)"].isnull()].copy()
missing_house_prices["Count"] = 1 
missing_house_prices = missing_house_prices.count()
##1424 obs with missing median house prices 

In [116]:
ACS5_2005_2019_WESTd = ACS5_2005_2019_WESTd[ACS5_2005_2019_WESTd["Median value (dollars)"].notnull()].drop(columns = ["index"]).reset_index()
#select rows with no missing median house prices

In [123]:
cols = ['index','REGIONA','DIVISIONA','COUSUBA','PLACEA','CONCITA','Shape_Leng','Shape_Area']
ACS5_2005_2019_WESTd = ACS5_2005_2019_WESTd.drop(columns=cols)

In [131]:
rename_dict = {"Total Race":"race_0", 
               "White alone":"race_1",
              "Black or African American alone":"race_2",
              "American Indian and Alaska Native alone":"race_3",
              "Asian alone":"race_4",
              "Native Hawaiian and Other Pacific Islander alone":"race_5",
              "Some other race alone":"race_6",
              "Two or more races":"race_7",
              "Total Travel Time to Work":"travel_0",
              "Less than 5 minutes": "travel_1",
              "5 to 9 minutes":"travel_2",
              "10 to 14 minutes":"travel_3",
              "15 to 19 minutes":"travel_4",
              "20 to 24 minutes":"travel_5",
              "25 to 29 minutes":"travel_6",
              "30 to 34 minutes":"travel_7",
              "35 to 39 minutes":"travel_8",
              "40 to 44 minutes":"travel_9",
              "45 to 59 minutes":"travel_10",
              "60 to 89 minutes":"travel_11",
              "90 or more minutes":"travel_12",
              "Total Educational Attainment for the Population 25 Years and Over":"educ_0",
              "No schooling completed":"educ_1",
              "Nursery to 4th grade":"educ_2",
              "5th and 6th grade":"educ_3",
              "7th and 8th grade":"educ_4",
              "9th grade":"educ_5",
              "10th grade":"educ_6",
              "11th grade":"educ_7",
              "12th grade no diploma":"educ_8",
              "High school graduate, GED, or alternative":"educ_9",
              "Some college less than 1 year":"educ_10",
              "Some college 1 or more years no degree":"educ_11",
              "Associate's degree":"educ_12",
              "Bachelor's degree":"educ_13",
              "Master's degree":"educ_14",
              "Professional school degree":"educ_15",
              "Doctorate degree":"educ_16",
              "Total Year Structure Built":"built_0",
              "Built 2000 or later":"built_1",
              "Built 1990 to 1999":"built_2",
              "Built 1980 to 1989":"built_3",
              "Built 1970 to 1979":"built_4",
              "Built 1960 to 1969":"built_5",
              "Built 1950 to 1959":"built_6",
              "Built 1940 to 1949":"built_7",
              "Built 1939 or earlier":"built_8",
              "Total Bedrooms":"bdrm_0",
              "1 bedroom":"bdrm_1",
              "2 bedrooms":"bdrm_2",
              "3 bedrooms":"bdrm_3",
              "4 bedrooms":"bdrm_4",
              "5 or more bedrooms":"bdrm_5", 
              "Median gross rent":"rent",
              "Lower value quartile (dollars)":"lowval",
              "Median value (dollars)":"medval",
              "Upper value quartile (dollars)":"upval",
              "Total Mortgage Status":"mortag_0",
              "Housing units with a mortgage contract to purchase or similar debt":"mortag_1",
              "Housing units with a mortgage contract to purchase or similar debt With either a second mortgage or home equity loan but not both": "mortag_2",
              "Housing units with a mortgage contract to purchase or similar debt Both second mortgage and home equity loan":"mortag_3",
              "Housing units with a mortgage contract to purchase or similar debt No second mortgage and no home equity loan":"mortag_4",
              "Housing units without a mortgage":"mortag_5"}

In [132]:
ACS5_2005_2019_WESTd = ACS5_2005_2019_WESTd.rename(columns=rename_dict)

In [136]:
ACS5_2005_2019_WESTd = ACS5_2005_2019_WESTd.drop(columns=['geometry'])

In [139]:
#ACS5_2005_2019_WESTd.to_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\merged_data_with_block_groups\ACS5_2005_2019_FULL_DATAFRAME_FOR_ANALYSIS_no_geo.csv")